In [1]:
import fasttext
from huggingface_hub import hf_hub_download
import pandas as pd

In [2]:
data = pd.read_csv("../data/jobs.csv")
data

,title,introduction,description,profile,we_offer,gehalt_min_yearly,gehalt_max_yearly
0,(Junior) Data Scientist (m/w/d),Wir suchen in Voll- oder Teilzeit Wir suchen ...,Entwicklung und Implementierung von Algorith­m...,Berufserfahrung im oben genannten Aufgaben­ber...,Freie Hardwarewahl nach deinen Wünschen inkl. ...,48384,68712
1,Junior Data Scientist (m/w/d),TUI Cruises ist einer der führenden Kreuzfahrt...,"Du analysierst große, komplexe Datensätze mit ...",Bachelor Abschluss in (Wirtschafts-)Informatik...,Eine flexible Arbeitszeitkultur mit zwei Mobil...,42576,60972
2,Junior Data Scientist (m/f/x),TUI Cruises is one of the leading cruise provi...,"Analyze large, complex datasets to extract ins...","Bachelor’s degree in Computer Science, Data Sc...",A flexible working time culture with two mobil...,42576,60972
3,(Junior) Data Scientist / Data Analyst (w/m/d),Mit rund 15 Mio. Kundenverträgen und 3.200 Mit...,"Für unsere Abteilung ""Customer Decisioning"" su...",Gute Deutschkenntnisse in Wort und Schrift Er...,Attraktives Gehalt Vertrauensarbeitszeit mit ...,46452,73548
4,Junior Data Scientist (m/w/d) – Risikomodelle,Die Sparkassen Rating und Risikosysteme (SR) i...,Mitarbeit an Gremien- oder Kundenprojekten im ...,Abgeschlossenes Studium (ggf. Promotion) im Be...,Eine unbefristete Tätigkeit in einem hoch vers...,44520,61932
...,...,...,...,...,...,...,...
3598,(Junior) Analyst Anti-Financial Crime (AFC) (m...,Wir sind heute [movers] und morgen [shapers].A...,Du bringst Deine Erfahrung aus dem Studium / a...,Eine erfolgreich abgeschlossene Ausbildung zum...,Weiterbildung: umfangreiche fachliche und meth...,36780,52260
3599,(Junior) Process Analyst - Customer Service (m...,Wir sind eines der führenden Home-Healthcare-U...,Verantwortlicher Ansprechpartner und Koordinat...,"Bachelor in betriebswirtschaftlichem Bereich, ...",Ankommen leicht gemacht: Wir legen großen Wert...,44520,73548
3600,Structured Finance Data Analyst (m/f/d) Native...,Converting Data into Market Trust European Dat...,Develop and maintain close relationships with ...,Bachelor degree in Finance /Economics /Banking...,The opportunity to work in a data-driven marke...,41616,67740
3601,"Senior Business Analyst (m/w/d) Regulatory, Da...","HSBC Deutschland ist Teil der HSBC-Gruppe, ein...",Wir suchen Kollegen (m/w/d) mit Persönlichkeit...,Sie verfügen über ein erfolgreich abgeschlosse...,Work-Life Balance: Flexible Arbeitszeitmodelle...,62904,84192


In [3]:
iqr = data["gehalt_max_yearly"].quantile(0.75) - data["gehalt_max_yearly"].quantile(0.25)
upper_bound = data["gehalt_max_yearly"].quantile(0.75) + 1.5 * iqr
lower_bound = data["gehalt_min_yearly"].quantile(0.25) - 1.5 * iqr

no_outliers = data.loc[
    (data["gehalt_max_yearly"] < upper_bound)
    & (data["gehalt_min_yearly"] > lower_bound), :]

In [4]:
data.loc[:, "full_text"] =  (
    data["title"].fillna("") + " "
    + data["introduction"].fillna("") + " "
    + data["description"].fillna("") + " "
    + data["profile"].fillna("") + " "
    + data["we_offer"].fillna("")
).str.strip()

In [5]:
model_path = hf_hub_download(repo_id="facebook/fasttext-language-identification", filename="model.bin")
model = fasttext.load_model(model_path)

model.bin:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

In [9]:
prediction = model.predict(data.loc[0, "full_text"])
prediction[0][0]

'__label__deu_Latn'

In [10]:
def get_lang(text:str):
    prediction = model.predict(text)
    lang = prediction[0][0]
    return lang

In [11]:
data.loc[:, "lang"] = data["full_text"].apply(get_lang)
data.lang.value_counts(normalize=True)

lang
__label__deu_Latn    0.875937
__label__eng_Latn    0.124063
Name: proportion, dtype: float64

In [12]:
data.loc[data["lang"] == "__label__eng_Latn"]

,title,introduction,description,profile,we_offer,gehalt_min_yearly,gehalt_max_yearly,full_text,lang
2,Junior Data Scientist (m/f/x),TUI Cruises is one of the leading cruise provi...,"Analyze large, complex datasets to extract ins...","Bachelor’s degree in Computer Science, Data Sc...",A flexible working time culture with two mobil...,42576,60972,Junior Data Scientist (m/f/x) TUI Cruises is o...,__label__eng_Latn
22,Data Scientist (f/m/x),HOP ON YOUR NEXT ADVENTURE WITH US! We are ...,"Develop, improve, deploy, and maintain predic...",You have an academic degree in an analytical ...,We value your participation in Tourlane's suc...,43548,61932,Data Scientist (f/m/x) HOP ON YOUR NEXT ADVEN...,__label__eng_Latn
42,Senior Expert Data Scientist (m/f/d),Our team gives everything to make the relation...,Contribute to the development and execution of...,"Ph.D. in a relevant field (e.g., computer scie...",Your freedom: pro rata mobile working and flex...,60000,80328,Senior Expert Data Scientist (m/f/d) Our team ...,__label__eng_Latn
45,Data Scientist (m/f/d) Machine Learning InsurTech,CHECK24 is the leading digital comparison plat...,Build a strong understanding around the busine...,Successfully completed degree (e.g. data scien...,Best of both worlds: CHECK24 combines all bene...,61932,85164,Data Scientist (m/f/d) Machine Learning InsurT...,__label__eng_Latn
47,OpenAI Technical Expert / Data Scientist (f/m/d),Seit 1866 gilt unsere Leidenschaft der Technik...,Development and implementation of Machine Lear...,"Master in computer science, mathematics, physi...",In addition to a secure job in a collegial and...,55164,84192,OpenAI Technical Expert / Data Scientist (f/m/...,__label__eng_Latn
...,...,...,...,...,...,...,...,...,...
3506,"Technical Program Manager III, Data Center Con...",A problem isn’t truly solved until it’s solve...,"As a Technical Program Manager at Google, you...",Minimum qualifications: Bachelor's degree in ...,NaN,53220,82260,"Technical Program Manager III, Data Center Con...",__label__eng_Latn
3554,Junior Sustainability Regulations Analyst (f/m/x),"Are you looking for more than just a job, for ...",The Junior Analyst of Sustainability Regulatio...,University degree or equivalent qualification ...,We are an aspiring corporate group where respe...,45480,62904,Junior Sustainability Regulations Analyst (f/m...,__label__eng_Latn
3582,(Senior) Data Scientist (Data Scientist),THE COMPANYinne is the next generation of fem...,:• Be responsible for the continuous improvem...,ABOUT YOU• Programming Experience in Python a...,of female health! OFFER AND BENEFITS• Competi...,45480,70644,(Senior) Data Scientist (Data Scientist) THE ...,__label__eng_Latn
3584,Data Science Graduate Program,"Bertelsmann is a media, services and education...",Data Science Graduate Program (m/f/d)Apply by ...,Your future employer Bertelsmann is a media...,What we offer you* Global network and communit...,62904,67740,Data Science Graduate Program Bertelsmann is a...,__label__eng_Latn


In [15]:
german = data.drop(data[data["lang"] == "__label__eng_Latn"].index).drop(columns="lang").reset_index(drop=True)
german

,title,introduction,description,profile,we_offer,gehalt_min_yearly,gehalt_max_yearly,full_text
0,(Junior) Data Scientist (m/w/d),Wir suchen in Voll- oder Teilzeit Wir suchen ...,Entwicklung und Implementierung von Algorith­m...,Berufserfahrung im oben genannten Aufgaben­ber...,Freie Hardwarewahl nach deinen Wünschen inkl. ...,48384,68712,(Junior) Data Scientist (m/w/d) Wir suchen in ...
1,Junior Data Scientist (m/w/d),TUI Cruises ist einer der führenden Kreuzfahrt...,"Du analysierst große, komplexe Datensätze mit ...",Bachelor Abschluss in (Wirtschafts-)Informatik...,Eine flexible Arbeitszeitkultur mit zwei Mobil...,42576,60972,Junior Data Scientist (m/w/d) TUI Cruises ist ...
2,(Junior) Data Scientist / Data Analyst (w/m/d),Mit rund 15 Mio. Kundenverträgen und 3.200 Mit...,"Für unsere Abteilung ""Customer Decisioning"" su...",Gute Deutschkenntnisse in Wort und Schrift Er...,Attraktives Gehalt Vertrauensarbeitszeit mit ...,46452,73548,(Junior) Data Scientist / Data Analyst (w/m/d)...
3,Junior Data Scientist (m/w/d) – Risikomodelle,Die Sparkassen Rating und Risikosysteme (SR) i...,Mitarbeit an Gremien- oder Kundenprojekten im ...,Abgeschlossenes Studium (ggf. Promotion) im Be...,Eine unbefristete Tätigkeit in einem hoch vers...,44520,61932,Junior Data Scientist (m/w/d) – Risikomodelle ...
4,Junior Aktuar / Data Scientist für das Pricing...,Weltweit vertrauen viele Kunden der Allianz Gr...,Aufgaben: Entwicklung von stochastischen Prog...,"Du suchst eine spannende, herausfordernde Täti...",NaN,48384,83220,Junior Aktuar / Data Scientist für das Pricing...
...,...,...,...,...,...,...,...,...
3151,Supply Chain Planner/Data Analyst (d/m/w),GROHE ist eine führende globale Marke für ganz...,Sie wollen Dinge bewegen und aktiv weiterentwi...,Unsere Anforderungen an Sie:Abgeschlossenes St...,Wir bieten Ihnen:Diversity & Inclusion: Bei un...,45480,68712,Supply Chain Planner/Data Analyst (d/m/w) GROH...
3152,(Junior) Analyst Anti-Financial Crime (AFC) (m...,Wir sind heute [movers] und morgen [shapers].A...,Du bringst Deine Erfahrung aus dem Studium / a...,Eine erfolgreich abgeschlossene Ausbildung zum...,Weiterbildung: umfangreiche fachliche und meth...,36780,52260,(Junior) Analyst Anti-Financial Crime (AFC) (m...
3153,(Junior) Process Analyst - Customer Service (m...,Wir sind eines der führenden Home-Healthcare-U...,Verantwortlicher Ansprechpartner und Koordinat...,"Bachelor in betriebswirtschaftlichem Bereich, ...",Ankommen leicht gemacht: Wir legen großen Wert...,44520,73548,(Junior) Process Analyst - Customer Service (m...
3154,"Senior Business Analyst (m/w/d) Regulatory, Da...","HSBC Deutschland ist Teil der HSBC-Gruppe, ein...",Wir suchen Kollegen (m/w/d) mit Persönlichkeit...,Sie verfügen über ein erfolgreich abgeschlosse...,Work-Life Balance: Flexible Arbeitszeitmodelle...,62904,84192,"Senior Business Analyst (m/w/d) Regulatory, Da..."


In [17]:
test = german.sample(frac=0.2)
train = german.drop(test.index)

In [20]:
test.to_csv("../data/test.csv", index=False)
train.to_csv("../data/train.csv", index=False)

In [19]:
train

,title,introduction,description,profile,we_offer,gehalt_min_yearly,gehalt_max_yearly,full_text
0,(Junior) Data Scientist (m/w/d),Wir suchen in Voll- oder Teilzeit Wir suchen ...,Entwicklung und Implementierung von Algorith­m...,Berufserfahrung im oben genannten Aufgaben­ber...,Freie Hardwarewahl nach deinen Wünschen inkl. ...,48384,68712,(Junior) Data Scientist (m/w/d) Wir suchen in ...
1,Junior Data Scientist (m/w/d),TUI Cruises ist einer der führenden Kreuzfahrt...,"Du analysierst große, komplexe Datensätze mit ...",Bachelor Abschluss in (Wirtschafts-)Informatik...,Eine flexible Arbeitszeitkultur mit zwei Mobil...,42576,60972,Junior Data Scientist (m/w/d) TUI Cruises ist ...
2,(Junior) Data Scientist / Data Analyst (w/m/d),Mit rund 15 Mio. Kundenverträgen und 3.200 Mit...,"Für unsere Abteilung ""Customer Decisioning"" su...",Gute Deutschkenntnisse in Wort und Schrift Er...,Attraktives Gehalt Vertrauensarbeitszeit mit ...,46452,73548,(Junior) Data Scientist / Data Analyst (w/m/d)...
3,Junior Data Scientist (m/w/d) – Risikomodelle,Die Sparkassen Rating und Risikosysteme (SR) i...,Mitarbeit an Gremien- oder Kundenprojekten im ...,Abgeschlossenes Studium (ggf. Promotion) im Be...,Eine unbefristete Tätigkeit in einem hoch vers...,44520,61932,Junior Data Scientist (m/w/d) – Risikomodelle ...
4,Junior Aktuar / Data Scientist für das Pricing...,Weltweit vertrauen viele Kunden der Allianz Gr...,Aufgaben: Entwicklung von stochastischen Prog...,"Du suchst eine spannende, herausfordernde Täti...",NaN,48384,83220,Junior Aktuar / Data Scientist für das Pricing...
...,...,...,...,...,...,...,...,...
3150,Junior Business Analyst (m/w/d) Sachversicherung,Dortmundunbefristet Vollzeit,Als Business Analyst in der Sachversicherung f...,Ausbildung: Sie haben eine Ausbildung zum Kauf...,Betriebliche Altersvorsorge Heute an übermorg...,49356,64836,Junior Business Analyst (m/w/d) Sachversicheru...
3151,Supply Chain Planner/Data Analyst (d/m/w),GROHE ist eine führende globale Marke für ganz...,Sie wollen Dinge bewegen und aktiv weiterentwi...,Unsere Anforderungen an Sie:Abgeschlossenes St...,Wir bieten Ihnen:Diversity & Inclusion: Bei un...,45480,68712,Supply Chain Planner/Data Analyst (d/m/w) GROH...
3153,(Junior) Process Analyst - Customer Service (m...,Wir sind eines der führenden Home-Healthcare-U...,Verantwortlicher Ansprechpartner und Koordinat...,"Bachelor in betriebswirtschaftlichem Bereich, ...",Ankommen leicht gemacht: Wir legen großen Wert...,44520,73548,(Junior) Process Analyst - Customer Service (m...
3154,"Senior Business Analyst (m/w/d) Regulatory, Da...","HSBC Deutschland ist Teil der HSBC-Gruppe, ein...",Wir suchen Kollegen (m/w/d) mit Persönlichkeit...,Sie verfügen über ein erfolgreich abgeschlosse...,Work-Life Balance: Flexible Arbeitszeitmodelle...,62904,84192,"Senior Business Analyst (m/w/d) Regulatory, Da..."
